In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from census import Census
import requests
import json
from uszipcode import SearchEngine
from api_keys import api_key

# Import census key and set to year 2016
c = Census("8d266861f3d4e6a17282942ceee2203290990edd", year=2016)

In [2]:
# Run Census Search to retrieve data on by states
census_data = c.acs5.get(("NAME", "B01003_001E", "B17001_002E", "B15003_002E","B15003_002E", 
                          "B15003_017E", "B15003_018E", "B15003_021E", "B15003_022E", "B15003_023E", 
                          "B15003_024E", "B15003_025E"), {'for': 'zip code tabulation area:*'})

# Create datafram
census_df = pd.DataFrame(census_data)

# Relabel and Reorder Columns
census_df = census_df.rename(columns={"zip code tabulation area": "Zipcode",
                                      "NAME": "Name",
                                      "B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B15003_002E": "No Education", 
                                      "B15003_017E": "High School Education", 
                                      "B15003_018E": "GED Education",
                                      "B15003_021E": "Associates Degree",
                                      "B15003_022E": "Bachelors Degree", 
                                      "B15003_023E": "Masters Degree",
                                      "B15003_024E": "Professional Degree",
                                      "B15003_025E": "Doctoral Degree"
                                      })

census_df = census_df.reindex(columns=["Zipcode", "Population", "Poverty Count",  "No Education", "High School Education", 
                                       "GED Education", "Associates Degree", "Bachelors Degree" , "Masters Degree", 
                                       "Professional Degree", "Doctoral Degree"])

# Print
census_df.head()

,Zipcode,Population,Poverty Count,No Education,High School Education,GED Education,Associates Degree,Bachelors Degree,Masters Degree,Professional Degree,Doctoral Degree
0,83202,23192.0,3504.0,108.0,3432.0,688.0,1539.0,2131.0,636.0,97.0,166.0
1,83203,279.0,83.0,0.0,34.0,28.0,13.0,16.0,0.0,0.0,0.0
2,83204,18071.0,3781.0,69.0,2654.0,698.0,866.0,2212.0,661.0,324.0,295.0
3,83209,1165.0,174.0,12.0,0.0,0.0,14.0,32.0,25.0,0.0,13.0
4,83210,3175.0,409.0,45.0,541.0,86.0,143.0,117.0,73.0,0.0,5.0


In [3]:
# Create No Degree column
census_df["No Degree"] = census_df["No Education"] + census_df["High School Education"] + census_df["GED Education"]

# Create Degree column
census_df["Degree"] = census_df["Associates Degree"] + census_df["Bachelors Degree"] + census_df["Masters Degree"] \
                    + census_df["Professional Degree"] + census_df["Doctoral Degree"]

# All columns needed for data by state saved to .csv in case needed
census_df.to_csv("Census_by_zip_all.csv", index=False)

# Print
census_df.head()

,Zipcode,Population,Poverty Count,No Education,High School Education,GED Education,Associates Degree,Bachelors Degree,Masters Degree,Professional Degree,Doctoral Degree,No Degree,Degree
0,83202,23192.0,3504.0,108.0,3432.0,688.0,1539.0,2131.0,636.0,97.0,166.0,4228.0,4569.0
1,83203,279.0,83.0,0.0,34.0,28.0,13.0,16.0,0.0,0.0,0.0,62.0,29.0
2,83204,18071.0,3781.0,69.0,2654.0,698.0,866.0,2212.0,661.0,324.0,295.0,3421.0,4358.0
3,83209,1165.0,174.0,12.0,0.0,0.0,14.0,32.0,25.0,0.0,13.0,12.0,84.0
4,83210,3175.0,409.0,45.0,541.0,86.0,143.0,117.0,73.0,0.0,5.0,672.0,338.0


In [4]:
# Only use necessary columns
census_df = census_df.reindex(columns=["Zipcode", "Population", "Poverty Count", "No Degree", "Degree"])
census_df.head()

,Zipcode,Population,Poverty Count,No Degree,Degree
0,83202,23192.0,3504.0,4228.0,4569.0
1,83203,279.0,83.0,62.0,29.0
2,83204,18071.0,3781.0,3421.0,4358.0
3,83209,1165.0,174.0,12.0,84.0
4,83210,3175.0,409.0,672.0,338.0


In [5]:
# Create new columns for Lat on Lng
census_df["State"] = ""
census_df["County"] = ""

census_df.head()

,Zipcode,Population,Poverty Count,No Degree,Degree,State,County
0,83202,23192.0,3504.0,4228.0,4569.0,,
1,83203,279.0,83.0,62.0,29.0,,
2,83204,18071.0,3781.0,3421.0,4358.0,,
3,83209,1165.0,174.0,12.0,84.0,,
4,83210,3175.0,409.0,672.0,338.0,,


In [6]:
# Add search from uszipcode dependency
search = SearchEngine(simple_zipcode=True)

# Add city of each zipcode
for index, row in census_df.iterrows():
    target_zip = row["Zipcode"]
    zipcode = search.by_zipcode(target_zip)
    
    try:
        census_df.loc[index, "State"] = zipcode.to_dict()['state']
        census_df.loc[index, "County"] = zipcode.to_dict()['county']
                
    except (KeyError):
        print("Zip not found. Skipping...")
        
census_df.head()

,Zipcode,Population,Poverty Count,No Degree,Degree,State,County
0,83202,23192.0,3504.0,4228.0,4569.0,ID,Bannock County
1,83203,279.0,83.0,62.0,29.0,ID,Bannock County
2,83204,18071.0,3781.0,3421.0,4358.0,ID,Bannock County
3,83209,1165.0,174.0,12.0,84.0,ID,Bannock County
4,83210,3175.0,409.0,672.0,338.0,ID,Bingham County


In [7]:
# Just pull California
california_zips = census_df.loc[census_df["State"] == "CA"]
california_zips

,Zipcode,Population,Poverty Count,No Degree,Degree,State,County
125,92231,40070.0,10421.0,6124.0,4648.0,CA,Imperial County
126,92233,7770.0,1188.0,1747.0,377.0,CA,Imperial County
127,92234,53253.0,11581.0,11090.0,8676.0,CA,Riverside County
128,92236,44124.0,13186.0,8796.0,1353.0,CA,Riverside County
129,92239,196.0,56.0,58.0,16.0,CA,Riverside County
130,92240,35684.0,11362.0,7411.0,4092.0,CA,Riverside County
131,92241,9212.0,2228.0,2302.0,1565.0,CA,Riverside County
132,92242,1332.0,242.0,410.0,171.0,CA,San Bernardino County
133,92243,49425.0,11666.0,6916.0,7558.0,CA,Imperial County
134,92249,7960.0,1095.0,1215.0,654.0,CA,Imperial County


In [8]:
del california_zips['Zipcode']
california_zips = california_zips.groupby("County").sum()
california_zips = california_zips.reset_index()
california_zips["Lat"] = ""
california_zips["Lng"] = ""
california_zips

,County,Population,Poverty Count,No Degree,Degree,Lat,Lng
0,Alameda County,1611207.0,189954.0,227485.0,566876.0,,
1,Alpine County,1143.0,216.0,251.0,271.0,,
2,Amador County,37043.0,3723.0,7986.0,8736.0,,
3,Butte County,224522.0,46625.0,35811.0,51902.0,,
4,Calaveras County,44814.0,5631.0,9513.0,10558.0,,
5,Colusa County,21580.0,2932.0,3360.0,3278.0,,
6,Contra Costa County,1101740.0,111453.0,148507.0,361261.0,,
7,Del Norte County,27628.0,5219.0,6173.0,4550.0,,
8,El Dorado County,182965.0,17800.0,29480.0,56757.0,,
9,Fresno County,968011.0,255735.0,159229.0,162493.0,,


In [9]:
# Use openweathermap API to import Lat and Lng of Cities
base_url = "http://api.openweathermap.org/data/2.5/weather?q="

for index, row in california_zips.iterrows():
    target_county = row["County"]
    
    query = f"{base_url}{target_county}&appid={api_key}"
    response = requests.get(query).json()
    
    try:
        california_zips.loc[index, "Lat"] = response["coord"]["lat"]
        california_zips.loc[index, "Lng"] = response["coord"]["lon"]

    except (KeyError):
        pass

In [11]:
# Delete blank rows
california_zips = california_zips[california_zips.County != '']
california_zips = california_zips[california_zips.Lat != '']
california_zips

,County,Population,Poverty Count,No Degree,Degree,Lat,Lng
0,Alameda County,1611207.0,189954.0,227485.0,566876.0,37.6,-121.88
1,Alpine County,1143.0,216.0,251.0,271.0,38.6,-119.8
2,Amador County,37043.0,3723.0,7986.0,8736.0,38.45,-120.65
3,Butte County,224522.0,46625.0,35811.0,51902.0,39.65,-121.57
4,Calaveras County,44814.0,5631.0,9513.0,10558.0,38.17,-120.58
5,Colusa County,21580.0,2932.0,3360.0,3278.0,39.18,-122.27
6,Contra Costa County,1101740.0,111453.0,148507.0,361261.0,37.93,-121.93
7,Del Norte County,27628.0,5219.0,6173.0,4550.0,41.7,-123.9
8,El Dorado County,182965.0,17800.0,29480.0,56757.0,38.77,-120.52
9,Fresno County,968011.0,255735.0,159229.0,162493.0,36.67,-119.83


In [12]:
# New columns needed for data by county saved to .csv in case needed
# CSV is used in Map_by_County
california_zips.to_csv("california_zips.csv", index=False)